In [1]:
from env.workload import Workload, Workflow
from schedule.taskNcpEnv import TaskAndNcpEnvironment
from schedule.schedule import Scheduler
from model.hyperparameter import get_args
from model.dqn import DQN
from model.sparse_gat import SpGAT
import datetime
import time
import pickle
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from load_balancing.server_client import *

args = get_args()
args.debug = False

In [2]:
def train(agent, train_wf_path, save_model_path,model, xhn_works, xhn_nodes):
  mean_makespan = []
  mean_cost = []
  load_balance = []
  time_rate = []
  cost_rate = []
  success_both_rate = []

  model.train()
  agent.net.train(True)
  scheduler = Scheduler(agent, model, args)
  print("start at:", str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
  start = time.time()
        
  for episode in range(1, args.episode_number+1):
    Workflow.reset()
    Workload.reset()
  
    print("episode:",episode,"="*70)
    t, lb, tr, dcsm = TaskAndNcpEnvironment(train_wf_path, scheduler, episode*10, args, method="dqn", xhn_works=xhn_works, xhn_nodes=xhn_nodes);
    mean_makespan.append(t)
    load_balance.append(lb)
    time_rate.append(tr)
        
  s = str(datetime.timedelta(seconds=time.time()-start))
  print("total train time:", s)
  scheduler.trainPlotFinal(mean_makespan, load_balance, time_rate)
  scheduler.save_hyperparameters(args)
  agent.save_model(save_model_path)

In [3]:
def deal_tasks_hr(xhn_works, xhn_nodes):
    train_generator = 'generator'
    save_model_path_generator = '_generator.pth'

    model = SpGAT(args.nfeat, args.hidden, args.out_feature, args.dropout, args.alpha, args.nb_heads).to(args.device)

    agent1 = DQN(args, model)
    train(agent1, train_generator, save_model_path_generator, model, xhn_works, xhn_nodes)

In [4]:
import csv
import ast
csv_file_path = './task_data.csv'
csv_file_path2 = './ncp_data.csv'

class Node:
    def __init__(self, name, deal_speed, left_bandwidth, cpu_used_rate):
        self.name = name
        self.deal_speed = deal_speed
        self.left_bandwidth = left_bandwidth
        self.cpu_used_rate = cpu_used_rate


# 打开CSV文件并读取数据
with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    xhn_works = []
    works = csv.reader(csvfile)
    for row in works:
        # 对每行中的每个字符串进行解析
        parsed_row = []
        for str_item in row:
            try:
                # 使用 ast.literal_eval 转换为 Python 对象
                parsed_item = ast.literal_eval(str_item)
                parsed_row.append(parsed_item)
            except SyntaxError:
                print(f"无法解析字符串: {str_item}")
        xhn_works.append(parsed_row)

with open(csv_file_path2, 'r', encoding='utf-8') as csvfile:
    xhn_nodes = []
    nodes = csv.reader(csvfile)
    for row in nodes:
        node = Node(row[0], float(row[1]), int(row[2]), float(row[3]))
        xhn_nodes.append(node)
        if len(xhn_nodes) == args.action_num:
            break

deal_tasks_hr(xhn_works, xhn_nodes[0:5])

start at: 2025-04-26 19:57:04
episode: 1 ======================================================================
[297360.00 - Releaser  ] vm192.168.1.7:60005 virtual machine is released. vm tasks num = 21. 
[297360.00 - Releaser  ] vm192.168.1.7:60004 virtual machine is released. vm tasks num = 20. 
[297360.00 - Releaser  ] vm192.168.1.7:60011 virtual machine is released. vm tasks num = 20. 
[297360.00 - Releaser  ] vm192.168.1.7:60009 virtual machine is released. vm tasks num = 19. 
[306720.00 - Releaser  ] vm192.168.1.7:60001 virtual machine is released. vm tasks num = 20. 
[Deadline] wf.path: , wf.deadline = 73731.84, wf.makespan = 119846.80, wf.waiting_time = 304309.92, distance = -46114.96, num = 12
[Deadline] wf.path: , wf.deadline = 46082.40, wf.makespan = 147505.94, wf.waiting_time = 322697.24, distance = -101423.54, num = 12
[Deadline] wf.path: , wf.deadline = 46082.40, wf.makespan = 175169.30, wf.waiting_time = 341359.85, distance = -129086.90, num = 12
[Deadline] wf.path: , w

KeyboardInterrupt: 